In [ ]:
import numpy as np
import re
from itertools import islice
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy.constants as const

import os
import sys
from pathlib import Path
from matplotlib import style
from lmfit import Parameters, fit_report, minimize
from lmfit.models import LinearModel
style.use('default')                # needed for vscode dark theme to see the axis

: 

In [ ]:
save_as_pgf = 0
save_as_pdf = 1

if save_as_pgf == 1:
    #mpl.use("pgf")                        
    mpl.rc('text', usetex=True)           # needed for latex
    mpl.rc('pgf', texsystem='pdflatex')   # change this if using xetex or luatex
    mpl.rc('pgf', rcfonts=False)          # don't setup fonts from rc parameters
    mpl.rc('pgf', preamble="\n".join([    # load additional packages
           r"\usepackage[utf8x]{inputenc}",
           r"\usepackage[T1]{fontenc}",
           r"\usepackage{amsmath,amsfonts,amsthm,amssymb,amstext}"]))
    mpl.rc('font', size=8)                # default font size
    mpl.rc('font', family='serif')          # use serif fonts
    #mpl.rc('font', family='sans-serif')   # use sans-serif fonts
    mpl.rc('font', serif='Latin Modern')  # use latex default serif font
    mpl.rc('axes', titlesize=10)           # fontsize of the axes title
    mpl.rc('axes', titlepad=2)            # pad between axis and title
    mpl.rc('axes', labelsize=10)           # fontsize of the x and y labels
    mpl.rc('xtick', labelsize=8)          # fontsize of the tick labels
    mpl.rc('ytick', labelsize=8)          # fontsize of the tick labels
    mpl.rc('legend', fontsize=8)          # legend fontsize
    mpl.rc('figure', titlesize=10)         # fontsize of the figure title
    mpl.rc('lines', markersize=2)         # default line markersize
    mpl.rc('lines', marker="")            # default line marker
    mpl.rc('grid', alpha=0.4)             # transparency of the grid
    mpl.rc('grid', color="gray")          # color of the grid
    mpl.rc('grid', linewidth=0.5)         # linewidth of the grid
    mpl.rcParams['figure.constrained_layout.use'] = True # use constrained_layout
    mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b", "g", "orange", "purple"]) #matplotlib colororder-default ändern
    lw = 1.0                              # linewidth of the lines

    # width and height of tex document in inches
    # For Katrins Bachelor thesis
    # in Latex:
    # \usepackage{layouts}
    # textwidth in inches: \printinunitsof{in}\prntlen{\textwidth}
    
    fig_width = 6.202
    fig_height = fig_width * 0.6
    
elif save_as_pdf == 1:
    normal = 9                              # fontsize of the text
    mpl.rc('font', size=normal+2)             # default font size
    mpl.rc('font', family='serif')          # use serif fonts
    mpl.rc('axes', titlesize=normal)        # fontsize of the axes title
    mpl.rc('axes', titlepad=5)              # pad between axis and title
    mpl.rc('axes', labelpad=7)              # pad between axis and label
    mpl.rc('axes', labelsize=normal+2)      # fontsize of the x and y labels
    mpl.rc('xtick', labelsize=normal)       # fontsize of the tick labels
    mpl.rc('ytick', labelsize=normal)       # fontsize of the tick labels
    mpl.rc('legend', fontsize=normal)       # legend fontsize
    mpl.rc('figure', titlesize=normal+4)    # fontsize of the figure title
    mpl.rc('lines', markersize=12)          # default line markersize
    mpl.rc('lines', marker="")              # default line marker
    mpl.rc('grid', alpha=0.4)               # transparency of the grid              
    mpl.rc('grid', color="gray")            # color of the grid
    mpl.rc('grid', linewidth=0.5)           # linewidth of the grid
    mpl.rcParams["xtick.major.size"] = 8    # major tick size in points
    mpl.rcParams["ytick.major.size"] = 8    # major tick size in points
    mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b", "g", "orange", "purple"]) #matplotlib colororder-default ändern
    
    lw = 2
    ms_4 = 4
    ms_8 = 8

    # height and width for nice print in jupyter notebook
    fig_width = 8
    fig_height = 5
    
else:
    mpl.rc('font', size=16)                 # controls default text sizes
    mpl.rc('axes', titlesize=16)            # fontsize of the axes title
    mpl.rc('axes', labelsize=18)            # fontsize of the x and y labels
    mpl.rc('xtick', labelsize=16)           # fontsize of the tick labels
    mpl.rc('ytick', labelsize=16)           # fontsize of the tick labels
    mpl.rc('legend', fontsize=18)           # legend fontsize
    mpl.rc('figure', titlesize=18)          # fontsize of the figure title
    mpl.rc('grid', alpha=0.4)
    mpl.rc('lines', markersize=10)    
    
    

def save_plot_as_pgf(fig, path):
  if save_as_pgf == 1:
    fig.savefig(path, format="pgf")

def save_plot_as_pdf(fig, path):
  if save_as_pdf == 1:
    fig.savefig(path, format="pdf", bbox_inches='tight')


: 

## Haber-Bosch

In [ ]:
# load data: Haber_Bosch (HB)

with open('Haber_Bosch_E', 'r') as f:
    lines = f.readlines()

HB = np.loadtxt(lines, usecols=(1,))
# NH3
nh3_b2plyp = HB[1]
nh3_mp2 = HB[2]
nh3_b3lyp = HB[3]
nh3_ccsdt = HB[4]

#n2
n2_b2plyp = HB[6]
n2_mp2 = HB[7]
n2_b3lyp = HB[8]
n2_ccsdt = HB[9]

#h2
h2_b2plyp = HB[11]
h2_mp2 = HB[12]
h2_b3lyp = HB[13]
h2_ccsdt = HB[14]

#Enthalpy Corrections

with open('Enthalpy_correction', 'r') as f:
    lines = f.readlines()
HB_thermo_corr = np.loadtxt(lines, usecols=(1,))
# NH3
nh3_b2plyp_corr = HB_thermo_corr[1]
nh3_mp2_corr = HB_thermo_corr[2]
nh3_b3lyp_corr = HB_thermo_corr[3]
nh3_ccsdt_corr = HB_thermo_corr[4]

#n2
n2_b2plyp_corr = HB_thermo_corr[6]
n2_mp2_corr = HB_thermo_corr[7]
n2_b3lyp_corr = HB_thermo_corr[8]
n2_ccsdt_corr = HB_thermo_corr[9]

#h2
h2_b2plyp_corr = HB_thermo_corr[11]
h2_mp2_corr = HB_thermo_corr[12]
h2_b3lyp_corr = HB_thermo_corr[13]
h2_ccsdt_corr = HB_thermo_corr[14]

#calculate Reaction enthalpy = H(Products) - H (Educts) = 2 * H(Nh3) - H (N2) - 3 * H (h2) 

#b2plyp
H_reac_b2plyp = 2 * (nh3_b2plyp + nh3_b2plyp_corr) - 3 * (h2_b2plyp + h2_b2plyp_corr) - (n2_b2plyp + n2_b2plyp_corr)





#make table 

# convert to pandas dataframe and add column names
names = ['Haber-Bosch']
df = pd.DataFrame({'H': H_energy, 'F': F_energy}, index=names)
df.columns = pd.MultiIndex.from_product([['Reaction Enthalpies'], df.columns])


# print dataframe
display(round(df,4))

#convert table to latex
print(df.to_latex(float_format="%.4f", escape=False))

: 